In [1]:
# 2023-Apr-26
# Jupyter notebook for correcting channel drift, since 2002, in SI spectra.

In [2]:
import numpy as np
from numpy.random import default_rng
from numpy.polynomial import Polynomial as P
import time
# from scipy import special
import scipy as sci
import pandas as pd

from ograyspy_class import select_spectrum_from_folder_list
from spec_class import Spec

In [3]:
import plotly.graph_objects as go

In [4]:
# a spectrum pattern to find and test:

In [5]:

# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:
# a_pattern = 'SI11318.Chn'
a_pattern = 'SI00718.Chn'

In [6]:
from ograyspy_class import Ograyspy

In [7]:
ogra = Ograyspy(batch_mode=False, folder_to_find='Genie_Transfer/Si/SI2018')
print('Objeto ogra: Ograyspy criado.')
print(f'ogra.info_plat: {ogra.info_plat}')
print(f'ogra.info_mach: {ogra.info_mach}')
print(f'ogra.info_syst: {ogra.info_syst}')
print(f'ogra.info_node: {ogra.info_node}')
print(f'ogra.home_path: {ogra.home_path}')

Found folder name:  /home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2018
Partes:  ('/', 'home', 'marcelofm', 'Nextcloud', 'Genie_Transfer', 'Si', 'SI2018')
No. spec files:  58
Objeto ogra: Ograyspy criado.
ogra.info_plat: Linux-6.2.0-20-generic-x86_64-with-glibc2.37
ogra.info_mach: x86_64
ogra.info_syst: Linux
ogra.info_node: mmaduar-net13
ogra.home_path: /home/marcelofm


In [8]:
# Select a machine/system to read from:
spectra_list_df = pd.read_pickle(ogra.pkl_folder_files)
spectra_list_df

,info_plat,info_mach,info_syst,info_node,home_path,spectra_path,n_files,a_spec_ind,a_spec_name,reduced_f_name,files_list,reduced_names_files_list,spectra_pattern_names,pkl_folder_files
0,Linux-6.2.0-20-generic-x86_64-with-glibc2.37,x86_64,Linux,mmaduar-net13,/home/marcelofm,/home/marcelofm/Nextcloud/Genie_Transfer/Si/SI...,58,0,,,[/home/marcelofm/Nextcloud/Genie_Transfer/Si/S...,"[SI09918.Chn, SI10118.Chn, SI06818.Chn, SI0061...","[**/*.[Cc][Hh][Nn], **/*.[Ii][Ee][Cc]]",Linuxmmaduar-net13.pkl


In [9]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [10]:
a_spec_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)

Existing:
Achou! indice=12, nomearq = SI00718.Chn
name:  /home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2018/SI00718.Chn
Final choices:
spectra_path: /home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2018
a_spec_name: /home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2018/SI00718.Chn
reduced_f_name: SI00718.Chn


In [11]:
a_spec_name, reduced_f_name

(PosixPath('/home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2018/SI00718.Chn'),
 'SI00718.Chn')

In [12]:
a_spec = Spec(a_spec_name, reduced_f_name)

In [13]:
y0_s = np.array(a_spec.gross_spec_ser_an.y_s)
y0_s

array([0, 0, 0, ..., 2, 4, 0])

In [14]:
x0_s = np.array(a_spec.gross_spec_ser_an.x_s)
x0_s

array([0.000e+00, 1.000e+00, 2.000e+00, ..., 4.093e+03, 4.094e+03,
       4.095e+03])

In [15]:
gross_ser = a_spec.gross_spec_ser_an

In [16]:
x_nz = gross_ser.chans_nzero
y_nz = gross_ser.counts_nzero

In [17]:
unc_y = gross_ser.unc_y

In [18]:
# Seleciona sensibilidade dos picos do espectro de referência:
promin = 50

In [19]:
pks, _ = sci.signal.find_peaks(y0_s, prominence=promin)

In [20]:
fig2 = go.Figure(data=go.Scattergl(
    x=x_nz,
    y=y_nz,
    mode='markers'
))


In [21]:
fig2.add_trace(
    go.Scattergl(x=x0_s,
                 y=y0_s,
                 error_y=dict(
                     color='orange',width=3.0,
                     type='data', # value of error bar given in data coordinates
                     array=unc_y,
                     visible=True),
                 name="Counts & uncertaintes",
                 line=dict(color='orange',width=0.3)));

In [22]:
fig2.add_trace(
    go.Scattergl(x=pks,
                 y=y0_s[pks],
                 mode="markers",
                 name="peaks",
                 marker=dict(color='cyan',
                             symbol='circle',
                             size=15,
                             opacity=0.4,
                             line=dict(color='green', width=2.0)
                            )
                ));

In [23]:
# Set title and scale type    width=1800,

fig2.update_layout(
    title_text="Gamma-ray spectrum excerpt",
    height=900,
    margin=dict(l=10, r=10, t=30, b=10),
    paper_bgcolor="LightSteelBlue"
)
fig2.update_yaxes(type="log");
fig2.write_html('fig2.html', auto_open=True)


In [24]:
# Agora, o grande loop para comparar o espectro escolhido
# com todos os outros:

In [25]:
# Fazendo correlação entre cada espectro da pasta com um bem formado entre eles.
# Método 1: correlação direta

In [26]:
t = time.time()
# do stuff

In [27]:
i_n = 0
for i_sp, j_sp in zip(fil_lst, reduc_nms):
    a_spec = Spec(i_sp, j_sp)
    gross_ser = a_spec.gross_spec_ser_an
    y_s = gross_ser.y_s
    # convo = sci.signal.correlate(y_s,y_s)
    # convo = sci.signal.convolve(counts, counts)
    # convo = sci.signal.fftconvolve(y_s, y_s)
    # convo = sci.signal.fftconvolve(y_s, np.roll(y_s,1500))
    # shifted_y_s = np.roll(y_s,1319)
    convo = sci.signal.correlate(y_s, y0_s)
    max_arg = np.argmax(abs(convo))
    drift = y_s.size-max_arg-1
    cts_tot = np.sum(y_s)
    pks, _ = sci.signal.find_peaks(y_s, prominence=promin)
    n_pks = pks.size
    print(f'{i_n}: {j_sp}: {max_arg} ->  {drift}  n_pks: {n_pks}  Tot: {cts_tot}.')
    i_n += 1
    # af = sci.fft.fft(y_s)
    # bf = sci.fft.fft(shifted_y_s)
    # c = sci.fft.ifft(af * np.conj(bf))
    # time_shift = np.argmax(abs(c))

0: SI09918.Chn: 4091 ->  4  n_pks: 173  Tot: 1262821.
1: SI10118.Chn: 4090 ->  5  n_pks: 180  Tot: 1298342.
2: SI06818.Chn: 4093 ->  2  n_pks: 152  Tot: 1044219.
3: SI00618.Chn: 4095 ->  0  n_pks: 158  Tot: 1188093.
4: SI09618.Chn: 4110 ->  -15  n_pks: 161  Tot: 1149257.
5: Si05718.chn: 4088 ->  7  n_pks: 167  Tot: 1231378.
6: SI01018.Chn: 4096 ->  -1  n_pks: 147  Tot: 983400.
7: SI09618 (aprox 17ks).Chn: 4091 ->  4  n_pks: 76  Tot: 408212.
8: SI09218.Chn: 4092 ->  3  n_pks: 180  Tot: 1448714.
9: SI10518.Chn: 4086 ->  9  n_pks: 198  Tot: 1889651.
10: SI03118.Chn: 4024 ->  71  n_pks: 105  Tot: 593718.
11: Si03818.chn: 4072 ->  23  n_pks: 145  Tot: 4546990.
12: SI00718.Chn: 4095 ->  0  n_pks: 219  Tot: 2055617.
13: SI10018.Chn: 4096 ->  -1  n_pks: 169  Tot: 1385659.
14: SI09518.Chn: 4092 ->  3  n_pks: 164  Tot: 1406785.
15: SI08118.Chn: 4089 ->  6  n_pks: 209  Tot: 1973215.
16: SI02518.Chn: 4088 ->  7  n_pks: 16  Tot: 108440.
17: SI07718.Chn: 4103 ->  -8  n_pks: 204  Tot: 2214327.
18: ES

In [28]:
elapsd = time.time() - t
elapsd

2.4799673557281494

In [29]:
# 2023-Apr-27
# Interessante! A maioria dos espectros SI2018, em relação ao de referência (SI11318),
# Está com drift à direita entre 16 e 19.

# Escolhi então o SI00718, que é rico de picos e intenso (mais de 2 M-conts!!)

In [30]:
# Método 2: fft

In [31]:
t = time.time()
# do stuff

In [32]:
i_n = 0
conj_bf = np.conj(sci.fft.fft(y0_s))
for i_sp, j_sp in zip(fil_lst, reduc_nms):
    a_spec = Spec(i_sp, j_sp)
    gross_ser = a_spec.gross_spec_ser_an
    y_s = gross_ser.y_s
    # convo = sci.signal.correlate(y_s,y_s)
    # convo = sci.signal.convolve(counts, counts)
    # convo = sci.signal.fftconvolve(y_s, y_s)
    # convo = sci.signal.fftconvolve(y_s, np.roll(y_s,1500))
    # shifted_y_s = np.roll(y_s,1319)
    
    # convo = sci.signal.correlate(y_s, y0_s)
    # convo = sci.signal.fftconvolve(y_s, y0_s)
   
    af = sci.fft.fft(y_s)
    c = sci.fft.ifft(af * conj_bf)

    max_arg = np.argmax(abs(c))    
    drift = y_s.size-max_arg-1
    cts_tot = np.sum(y_s)
    pks, _ = sci.signal.find_peaks(y_s, prominence=promin)
    n_pks = pks.size
    print(f'{i_n}: {j_sp}: {max_arg} ->  {drift}  n_pks: {n_pks}  Tot: {cts_tot}.')
    i_n += 1
    # af = sci.fft.fft(y_s)
    # bf = sci.fft.fft(shifted_y_s)
    # c = sci.fft.ifft(af * np.conj(bf))
    # time_shift = np.argmax(abs(c))

0: SI09918.Chn: 4092 ->  3  n_pks: 173  Tot: 1262821.
1: SI10118.Chn: 4091 ->  4  n_pks: 180  Tot: 1298342.
2: SI06818.Chn: 4094 ->  1  n_pks: 152  Tot: 1044219.
3: SI00618.Chn: 0 ->  4095  n_pks: 158  Tot: 1188093.
4: SI09618.Chn: 15 ->  4080  n_pks: 161  Tot: 1149257.
5: Si05718.chn: 4089 ->  6  n_pks: 167  Tot: 1231378.
6: SI01018.Chn: 1 ->  4094  n_pks: 147  Tot: 983400.
7: SI09618 (aprox 17ks).Chn: 4092 ->  3  n_pks: 76  Tot: 408212.
8: SI09218.Chn: 4093 ->  2  n_pks: 180  Tot: 1448714.
9: SI10518.Chn: 4087 ->  8  n_pks: 198  Tot: 1889651.
10: SI03118.Chn: 4025 ->  70  n_pks: 105  Tot: 593718.
11: Si03818.chn: 4073 ->  22  n_pks: 145  Tot: 4546990.
12: SI00718.Chn: 0 ->  4095  n_pks: 219  Tot: 2055617.
13: SI10018.Chn: 1 ->  4094  n_pks: 169  Tot: 1385659.
14: SI09518.Chn: 4093 ->  2  n_pks: 164  Tot: 1406785.
15: SI08118.Chn: 4090 ->  5  n_pks: 209  Tot: 1973215.
16: SI02518.Chn: 4089 ->  6  n_pks: 16  Tot: 108440.
17: SI07718.Chn: 8 ->  4087  n_pks: 204  Tot: 2214327.
18: ESFREG

In [33]:
elapsd = time.time() - t
elapsd

2.4643962383270264

In [34]:
# Agora, fazer com todos da pasta SI!!!!!

In [35]:
del ogra

In [36]:
ogra = Ograyspy(batch_mode=False, folder_to_find='Genie_Transfer/Si')
print('Objeto ogra: Ograyspy criado.')
print(f'ogra.info_plat: {ogra.info_plat}')
print(f'ogra.info_mach: {ogra.info_mach}')
print(f'ogra.info_syst: {ogra.info_syst}')
print(f'ogra.info_node: {ogra.info_node}')
print(f'ogra.home_path: {ogra.home_path}')

Found folder name:  /home/marcelofm/Nextcloud/Genie_Transfer/Si
Partes:  ('/', 'home', 'marcelofm', 'Nextcloud', 'Genie_Transfer', 'Si')
No. spec files:  1009
Objeto ogra: Ograyspy criado.
ogra.info_plat: Linux-6.2.0-20-generic-x86_64-with-glibc2.37
ogra.info_mach: x86_64
ogra.info_syst: Linux
ogra.info_node: mmaduar-net13
ogra.home_path: /home/marcelofm


In [37]:
# Select a machine/system to read from:
spectra_list_df = pd.read_pickle(ogra.pkl_folder_files)
spectra_list_df

,info_plat,info_mach,info_syst,info_node,home_path,spectra_path,n_files,a_spec_ind,a_spec_name,reduced_f_name,files_list,reduced_names_files_list,spectra_pattern_names,pkl_folder_files
0,Linux-6.2.0-20-generic-x86_64-with-glibc2.37,x86_64,Linux,mmaduar-net13,/home/marcelofm,/home/marcelofm/Nextcloud/Genie_Transfer/Si,1009,0,,,[/home/marcelofm/Nextcloud/Genie_Transfer/Si/S...,"[Si2006/SI12306.Chn, Si2006/SI02606.Chn, Si200...","[**/*.[Cc][Hh][Nn], **/*.[Ii][Ee][Cc]]",Linuxmmaduar-net13.pkl


In [38]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [39]:
a_spec_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)

Existing:
Achou! indice=276, nomearq = SI2018/SI00718.Chn
Final choices:
spectra_path: /home/marcelofm/Nextcloud/Genie_Transfer/Si
a_spec_name: /home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2018/SI00718.Chn
reduced_f_name: SI2018/SI00718.Chn


In [40]:
a_spec_name, reduced_f_name

(PosixPath('/home/marcelofm/Nextcloud/Genie_Transfer/Si/SI2018/SI00718.Chn'),
 'SI2018/SI00718.Chn')

In [41]:
a_spec = Spec(a_spec_name, reduced_f_name)

In [42]:
y0_s = np.array(a_spec.gross_spec_ser_an.y_s)
y0_s

array([0, 0, 0, ..., 2, 4, 0])

In [43]:
x0_s = np.array(a_spec.gross_spec_ser_an.x_s)
x0_s

array([0.000e+00, 1.000e+00, 2.000e+00, ..., 4.093e+03, 4.094e+03,
       4.095e+03])

In [44]:
gross_ser = a_spec.gross_spec_ser_an

In [45]:
x_nz = gross_ser.chans_nzero
y_nz = gross_ser.counts_nzero

In [46]:
unc_y = gross_ser.unc_y

In [47]:
# Seleciona sensibilidade dos picos do espectro de referência:
promin = 50

In [48]:
pks, _ = sci.signal.find_peaks(y0_s, prominence=promin)

In [49]:
t = time.time()
# do stuff

In [50]:
i_n = 0
lista_mix = []
for i_sp, j_sp in zip(fil_lst, reduc_nms):
    a_spec = Spec(i_sp, j_sp)
    gross_ser = a_spec.gross_spec_ser_an
    y_s = gross_ser.y_s
    # convo = sci.signal.correlate(y_s,y_s)
    # convo = sci.signal.convolve(counts, counts)
    # convo = sci.signal.fftconvolve(y_s, y_s)
    # convo = sci.signal.fftconvolve(y_s, np.roll(y_s,1500))
    # shifted_y_s = np.roll(y_s,1319)
    convo = sci.signal.correlate(y_s, y0_s)
    max_arg = np.argmax(abs(convo))
    drift = y_s.size-max_arg-1
    cts_tot = np.sum(y_s)
    pks, _ = sci.signal.find_peaks(y_s, prominence=promin)
    n_pks = pks.size
    print(f'{i_n}: {j_sp}: {max_arg} ->  {drift}  n_pks: {n_pks}  Tot: {cts_tot}.')
    lista_mix.append([i_n, j_sp, max_arg, drift, n_pks, cts_tot])
    i_n += 1
    # af = sci.fft.fft(y_s)
    # bf = sci.fft.fft(shifted_y_s)
    # c = sci.fft.ifft(af * np.conj(bf))
    # time_shift = np.argmax(abs(c))

0: Si2006/SI12306.Chn: 4115 ->  -20  n_pks: 250  Tot: 2441964.
1: Si2006/SI02606.Chn: 4082 ->  13  n_pks: 18  Tot: 87979.
2: Si2006/SI16206.Chn: 4112 ->  -17  n_pks: 234  Tot: 2091009.
3: Si2006/si139.140.Chn: 4082 ->  13  n_pks: 50  Tot: 272363.
4: Si2006/si15906.Chn: 4072 ->  23  n_pks: 143  Tot: 939046.
5: Si2006/SI01506.Chn: 4113 ->  -18  n_pks: 239  Tot: 2228464.
6: Si2006/SI12506.Chn: 4112 ->  -17  n_pks: 185  Tot: 1620485.
7: Si2006/si00406.Chn: 4074 ->  21  n_pks: 152  Tot: 1050314.
8: Si2006/si05406.Chn: 4112 ->  -17  n_pks: 192  Tot: 1604868.
9: Si2006/si07206.Chn: 4084 ->  11  n_pks: 128  Tot: 879886.
10: Si2006/si16106.Chn: 4111 ->  -16  n_pks: 154  Tot: 1131034.
11: Si2006/si07706r.Chn: 4075 ->  20  n_pks: 82  Tot: 500686.
12: Si2006/si12906.Chn: 4112 ->  -17  n_pks: 239  Tot: 2401869.
13: Si2006/si02406.Chn: 4083 ->  12  n_pks: 53  Tot: 272024.
14: Si2006/SI10906.Chn: 4084 ->  11  n_pks: 37  Tot: 175986.
15: Si2006/SI13106.Chn: 4101 ->  -6  n_pks: 19  Tot: 96685.
16: Si20

134: SI2019/SI06519.Chn: 2713 ->  1382  n_pks: 68  Tot: 1374097.
135: SI2019/si00319.Chn: 4122 ->  -27  n_pks: 185  Tot: 1530163.
136: SI2019/si05919.Chn: 2918 ->  1177  n_pks: 51  Tot: 428293.
137: SI2019/SI06919.Chn: 4090 ->  5  n_pks: 180  Tot: 1431385.
138: SI2019/SI09919.Chn: 4088 ->  7  n_pks: 160  Tot: 1423231.
139: SI2019/SI05819.Chn: 4082 ->  13  n_pks: 217  Tot: 1876523.
140: SI2019/si09719.Chn: 2924 ->  1171  n_pks: 55  Tot: 1124564.
141: SI2019/SI5-19-2.Chn: 4100 ->  -5  n_pks: 192  Tot: 1817384.
142: SI2019/SI10419.Chn: 4096 ->  -1  n_pks: 45  Tot: 284564.
143: SI2019/si06019.Chn: 2700 ->  1395  n_pks: 55  Tot: 1450603.
144: SI2019/SI03419.Chn: 4096 ->  -1  n_pks: 161  Tot: 1226200.
145: SI2019/SI03919.Chn: 4103 ->  -8  n_pks: 18  Tot: 127565.
146: SI2007/si14307.Chn: 4083 ->  12  n_pks: 44  Tot: 338089.
147: SI2007/si01807.Chn: 4081 ->  14  n_pks: 9  Tot: 116590.
148: SI2007/si14507.Chn: 4081 ->  14  n_pks: 3  Tot: 42954.
149: SI2007/si03407.Chn: 4080 ->  15  n_pks: 14  T

265: SI2018/SI10118.Chn: 4090 ->  5  n_pks: 180  Tot: 1298342.
266: SI2018/SI06818.Chn: 4093 ->  2  n_pks: 152  Tot: 1044219.
267: SI2018/SI00618.Chn: 4095 ->  0  n_pks: 158  Tot: 1188093.
268: SI2018/SI09618.Chn: 4110 ->  -15  n_pks: 161  Tot: 1149257.
269: SI2018/Si05718.chn: 4088 ->  7  n_pks: 167  Tot: 1231378.
270: SI2018/SI01018.Chn: 4096 ->  -1  n_pks: 147  Tot: 983400.
271: SI2018/SI09618 (aprox 17ks).Chn: 4091 ->  4  n_pks: 76  Tot: 408212.
272: SI2018/SI09218.Chn: 4092 ->  3  n_pks: 180  Tot: 1448714.
273: SI2018/SI10518.Chn: 4086 ->  9  n_pks: 198  Tot: 1889651.
274: SI2018/SI03118.Chn: 4024 ->  71  n_pks: 105  Tot: 593718.
275: SI2018/Si03818.chn: 4072 ->  23  n_pks: 145  Tot: 4546990.
276: SI2018/SI00718.Chn: 4095 ->  0  n_pks: 219  Tot: 2055617.
277: SI2018/SI10018.Chn: 4096 ->  -1  n_pks: 169  Tot: 1385659.
278: SI2018/SI09518.Chn: 4092 ->  3  n_pks: 164  Tot: 1406785.
279: SI2018/SI08118.Chn: 4089 ->  6  n_pks: 209  Tot: 1973215.
280: SI2018/SI02518.Chn: 4088 ->  7  n_p

396: si2009/SI10309.Chn: 4112 ->  -17  n_pks: 239  Tot: 2331645.
397: si2009/si06209.Chn: 4102 ->  -7  n_pks: 190  Tot: 1690159.
398: si2009/si09909.Chn: 4115 ->  -20  n_pks: 282  Tot: 2725204.
399: si2009/SI09309.Chn: 4106 ->  -11  n_pks: 41  Tot: 238096.
400: si2009/SI09709.Chn: 4114 ->  -19  n_pks: 180  Tot: 1731633.
401: si2009/SI16509.Chn: 4119 ->  -24  n_pks: 179  Tot: 1831247.
402: si2009/SI10909.Chn: 4111 ->  -16  n_pks: 243  Tot: 2389452.
403: si2009/filtro3.Chn: 3190 ->  905  n_pks: 34  Tot: 165679.
404: si2009/SI03509.Chn: 4102 ->  -7  n_pks: 190  Tot: 1641572.
405: si2009/SI12109.Chn: 4099 ->  -4  n_pks: 186  Tot: 1305017.
406: si2009/SI16909.Chn: 4106 ->  -11  n_pks: 159  Tot: 1395867.
407: si2009/SI07609.Chn: 4121 ->  -26  n_pks: 214  Tot: 1865139.
408: si2009/SI06409.Chn: 4113 ->  -18  n_pks: 187  Tot: 1551822.
409: si2009/SI09109.Chn: 4115 ->  -20  n_pks: 145  Tot: 1147636.
410: si2009/SI04609.Chn: 4088 ->  7  n_pks: 71  Tot: 387650.
411: si2009/SI14409.Chn: 4103 ->  -8

528: SI2022/SI08422.Chn: 3012 ->  1083  n_pks: 106  Tot: 980701.
529: SI2022/SI05522.Chn: 4091 ->  4  n_pks: 79  Tot: 432149.
530: SI2022/SI02522.Chn: 4090 ->  5  n_pks: 83  Tot: 469313.
531: SI2022/SI05322.Chn: 4103 ->  -8  n_pks: 102  Tot: 563877.
532: SI2022/SI04722.Chn: 4103 ->  -8  n_pks: 97  Tot: 570053.
533: SI2022/SI07722.Chn: 4111 ->  -16  n_pks: 96  Tot: 517142.
534: SI2022/SI06122.Chn: 4079 ->  16  n_pks: 95  Tot: 548003.
535: SI2022/SI05922.Chn: 4106 ->  -11  n_pks: 93  Tot: 544534.
536: SI2022/SI07922.Chn: 4111 ->  -16  n_pks: 108  Tot: 566099.
537: SI2022/SI09322.Chn: 4117 ->  -22  n_pks: 90  Tot: 556587.
538: SI2022/SI08722.Chn: 4111 ->  -16  n_pks: 101  Tot: 539942.
539: SI2022/SI00822.Chn: 4089 ->  6  n_pks: 51  Tot: 398225.
540: SI2022/SI01722.Chn: 4091 ->  4  n_pks: 80  Tot: 479312.
541: SI2022/SI03322.Chn: 4105 ->  -10  n_pks: 81  Tot: 496478.
542: SI2022/SI03122.Chn: 4094 ->  1  n_pks: 82  Tot: 492151.
543: SI2022/SI08222.Chn: 4110 ->  -15  n_pks: 99  Tot: 535351.


659: si2011/si16111.Chn: 4098 ->  -3  n_pks: 167  Tot: 1306986.
660: si2011/SI00811.Chn: 4096 ->  -1  n_pks: 157  Tot: 1194959.
661: si2011/SI14411.Chn: 4116 ->  -21  n_pks: 190  Tot: 1830436.
662: si2011/si11011.Chn: 4111 ->  -16  n_pks: 247  Tot: 2701598.
663: si2011/ERFGA.Chn: 4114 ->  -19  n_pks: 380  Tot: 38551654.
664: si2011/SI07611.Chn: 4106 ->  -11  n_pks: 167  Tot: 1373843.
665: si2011/SI00411.chn: 4091 ->  4  n_pks: 112  Tot: 816905.
666: si2011/SI12911.Chn: 4115 ->  -20  n_pks: 196  Tot: 2048920.
667: si2011/SI13711.Chn: 4115 ->  -20  n_pks: 179  Tot: 1744233.
668: si2011/SI10511.Chn: 4113 ->  -18  n_pks: 112  Tot: 779306.
669: si2011/SI15611.Chn: 4113 ->  -18  n_pks: 158  Tot: 1289901.
670: si2011/si04911.Chn: 4118 ->  -23  n_pks: 219  Tot: 2890836.
671: si2011/SI12311.Chn: 4104 ->  -9  n_pks: 119  Tot: 739287.
672: si2011/SI01211.Chn: 4113 ->  -18  n_pks: 138  Tot: 1146308.
673: si2011/SI05311.Chn: 4110 ->  -15  n_pks: 173  Tot: 1915442.
674: si2011/SI01711.Chn: 4097 ->  

787: SI2004/si20304.chn: 4076 ->  19  n_pks: 208  Tot: 1670680.
788: SI2004/si11904.chn: 4117 ->  -22  n_pks: 237  Tot: 2291780.
789: SI2004/rqba.chn: 4084 ->  11  n_pks: 39  Tot: 175145.
790: SI2004/si034042.chn: 4092 ->  3  n_pks: 34  Tot: 313399.
791: SI2004/si18104.CHN: 4084 ->  11  n_pks: 95  Tot: 596138.
792: SI2004/SI13804.CHN: 4076 ->  19  n_pks: 144  Tot: 1016815.
793: SI2004/SI18604.CHN: 4074 ->  21  n_pks: 173  Tot: 1236513.
794: SI2004/SI14004.CHN: 4075 ->  20  n_pks: 161  Tot: 1182815.
795: SI2004/si20404.CHN: 4094 ->  1  n_pks: 163  Tot: 1199142.
796: SI2004/si15204.CHN: 4116 ->  -21  n_pks: 272  Tot: 2574971.
797: si2013/sr90_CR.Chn: 4109 ->  -14  n_pks: 146  Tot: 1543925.
798: si2013/si05713.Chn: 4108 ->  -13  n_pks: 156  Tot: 1358116.
799: si2013/SI01913.Chn: 4101 ->  -6  n_pks: 154  Tot: 1480186.
800: si2013/si06413.Chn: 4113 ->  -18  n_pks: 192  Tot: 2251832.
801: si2013/SI03913.Chn: 4108 ->  -13  n_pks: 154  Tot: 1233031.
802: si2013/SI01113.Chn: 4108 ->  -13  n_pks

917: Si2005/05005.CHN: 4072 ->  23  n_pks: 162  Tot: 1249410.
918: Si2005/si10005a.CHN: 4039 ->  56  n_pks: 62  Tot: 1054079.
919: Si2005/si16505.Chn: 4113 ->  -18  n_pks: 245  Tot: 2373853.
920: Si2005/si24505.Chn: 4112 ->  -17  n_pks: 179  Tot: 1480418.
921: Si2005/si16405.Chn: 4083 ->  12  n_pks: 33  Tot: 173021.
922: Si2005/si22105rr.Chn: 4038 ->  57  n_pks: 49  Tot: 415912.
923: Si2005/SI22405.Chn: 4039 ->  56  n_pks: 45  Tot: 250437.
924: Si2005/SI17305.Chn: 4082 ->  13  n_pks: 34  Tot: 166338.
925: Si2005/si09305.chn: 4084 ->  11  n_pks: 17  Tot: 83528.
926: Si2005/SI22205R.Chn: 4099 ->  -4  n_pks: 96  Tot: 667605.
927: Si2005/UNAT2.chn: 4083 ->  12  n_pks: 142  Tot: 964222.
928: Si2005/si06105.chn: 4083 ->  12  n_pks: 17  Tot: 77422.
929: Si2005/SI13105.CHN: 4084 ->  11  n_pks: 36  Tot: 181097.
930: Si2005/si18805.Chn: 4082 ->  13  n_pks: 25  Tot: 97842.
931: Si2005/si13005.CHN: 4084 ->  11  n_pks: 33  Tot: 189185.
932: Si2005/si14905.Chn: 4112 ->  -17  n_pks: 202  Tot: 1758298

In [51]:
elapsed = time.time() - t
elapsed

57.573267459869385

In [56]:
drifts = np.array([l[3] for l in lista_mix])
drifts

array([-20,  13, -17, ...,  -2, -35,  10])

In [57]:
# num=21 dá percentis 0%, 5%, 10%, ... 100%
percentis = np.percentile(drifts, np.linspace(0, 100, num=21))
percentis

array([-3.449e+03, -2.700e+01, -2.100e+01, -1.900e+01, -1.700e+01,
       -1.500e+01, -1.160e+01, -9.000e+00, -4.000e+00, -1.000e+00,
        3.000e+00,  6.000e+00,  7.000e+00,  9.000e+00,  1.100e+01,
        1.200e+01,  1.300e+01,  1.900e+01,  2.300e+01,  5.820e+01,
        1.450e+03])

In [58]:
a = np.array([0, 1, 2, 3, 4, 3, 2, 1, 0, 1, 2, 3, 4, 3, 2, 1, 0, 0, 0, 0, 0])
b = np.array([0, 0, 0, 0, 0, 1, 2, 3, 4, 3, 2, 1, 0, 1, 2, 3, 4, 3, 2, 1, 0])

In [59]:
a+np.roll(b, shift=-4)

array([0, 2, 4, 6, 8, 6, 4, 2, 0, 2, 4, 6, 8, 6, 4, 2, 0, 0, 0, 0, 0])

In [60]:
# Tcharannnn
# Soma de 20 anos de espectros da piscina!!!

In [83]:
i_n = 0
i_m = 0
ys_tot = np.zeros(y0_s.size)
for i_sp, j_sp in zip(fil_lst, reduc_nms):
    a_spec = Spec(i_sp, j_sp)
    gross_ser = a_spec.gross_spec_ser_an
    y_s = gross_ser.y_s
    convo = sci.signal.correlate(y_s, y0_s)
    max_arg = np.argmax(abs(convo))
    drift = y_s.size-max_arg-1
    cts_tot = np.sum(y_s)
    pks, _ = sci.signal.find_peaks(y_s, prominence=promin)
    n_pks = pks.size
    print(f'{i_n}: {j_sp}: {max_arg} ->  {drift}  n_pks: {n_pks}  Tot: {cts_tot}.')
    if abs(drift) < 60:
        # ys_tot += np.roll(y_s, -drift)
        ys_tot += y_s
        i_m += 1
    i_n += 1
    # af = sci.fft.fft(y_s)
    # bf = sci.fft.fft(shifted_y_s)
    # c = sci.fft.ifft(af * np.conj(bf))
    # time_shift = np.argmax(abs(c))

0: Si2006/SI12306.Chn: 4115 ->  -20  n_pks: 250  Tot: 2441964.
1: Si2006/SI02606.Chn: 4082 ->  13  n_pks: 18  Tot: 87979.
2: Si2006/SI16206.Chn: 4112 ->  -17  n_pks: 234  Tot: 2091009.
3: Si2006/si139.140.Chn: 4082 ->  13  n_pks: 50  Tot: 272363.
4: Si2006/si15906.Chn: 4072 ->  23  n_pks: 143  Tot: 939046.
5: Si2006/SI01506.Chn: 4113 ->  -18  n_pks: 239  Tot: 2228464.
6: Si2006/SI12506.Chn: 4112 ->  -17  n_pks: 185  Tot: 1620485.
7: Si2006/si00406.Chn: 4074 ->  21  n_pks: 152  Tot: 1050314.
8: Si2006/si05406.Chn: 4112 ->  -17  n_pks: 192  Tot: 1604868.
9: Si2006/si07206.Chn: 4084 ->  11  n_pks: 128  Tot: 879886.
10: Si2006/si16106.Chn: 4111 ->  -16  n_pks: 154  Tot: 1131034.
11: Si2006/si07706r.Chn: 4075 ->  20  n_pks: 82  Tot: 500686.
12: Si2006/si12906.Chn: 4112 ->  -17  n_pks: 239  Tot: 2401869.
13: Si2006/si02406.Chn: 4083 ->  12  n_pks: 53  Tot: 272024.
14: Si2006/SI10906.Chn: 4084 ->  11  n_pks: 37  Tot: 175986.
15: Si2006/SI13106.Chn: 4101 ->  -6  n_pks: 19  Tot: 96685.
16: Si20

135: SI2019/si00319.Chn: 4122 ->  -27  n_pks: 185  Tot: 1530163.
136: SI2019/si05919.Chn: 2918 ->  1177  n_pks: 51  Tot: 428293.
137: SI2019/SI06919.Chn: 4090 ->  5  n_pks: 180  Tot: 1431385.
138: SI2019/SI09919.Chn: 4088 ->  7  n_pks: 160  Tot: 1423231.
139: SI2019/SI05819.Chn: 4082 ->  13  n_pks: 217  Tot: 1876523.
140: SI2019/si09719.Chn: 2924 ->  1171  n_pks: 55  Tot: 1124564.
141: SI2019/SI5-19-2.Chn: 4100 ->  -5  n_pks: 192  Tot: 1817384.
142: SI2019/SI10419.Chn: 4096 ->  -1  n_pks: 45  Tot: 284564.
143: SI2019/si06019.Chn: 2700 ->  1395  n_pks: 55  Tot: 1450603.
144: SI2019/SI03419.Chn: 4096 ->  -1  n_pks: 161  Tot: 1226200.
145: SI2019/SI03919.Chn: 4103 ->  -8  n_pks: 18  Tot: 127565.
146: SI2007/si14307.Chn: 4083 ->  12  n_pks: 44  Tot: 338089.
147: SI2007/si01807.Chn: 4081 ->  14  n_pks: 9  Tot: 116590.
148: SI2007/si14507.Chn: 4081 ->  14  n_pks: 3  Tot: 42954.
149: SI2007/si03407.Chn: 4080 ->  15  n_pks: 14  Tot: 121576.
150: SI2007/SI06107.Chn: 4084 ->  11  n_pks: 21  Tot:

270: SI2018/SI01018.Chn: 4096 ->  -1  n_pks: 147  Tot: 983400.
271: SI2018/SI09618 (aprox 17ks).Chn: 4091 ->  4  n_pks: 76  Tot: 408212.
272: SI2018/SI09218.Chn: 4092 ->  3  n_pks: 180  Tot: 1448714.
273: SI2018/SI10518.Chn: 4086 ->  9  n_pks: 198  Tot: 1889651.
274: SI2018/SI03118.Chn: 4024 ->  71  n_pks: 105  Tot: 593718.
275: SI2018/Si03818.chn: 4072 ->  23  n_pks: 145  Tot: 4546990.
276: SI2018/SI00718.Chn: 4095 ->  0  n_pks: 219  Tot: 2055617.
277: SI2018/SI10018.Chn: 4096 ->  -1  n_pks: 169  Tot: 1385659.
278: SI2018/SI09518.Chn: 4092 ->  3  n_pks: 164  Tot: 1406785.
279: SI2018/SI08118.Chn: 4089 ->  6  n_pks: 209  Tot: 1973215.
280: SI2018/SI02518.Chn: 4088 ->  7  n_pks: 16  Tot: 108440.
281: SI2018/SI07718.Chn: 4103 ->  -8  n_pks: 204  Tot: 2214327.
282: SI2018/ESFREGAÇO CR MALVINA.Chn: 2964 ->  1131  n_pks: 40  Tot: 212328.
283: SI2018/SI08818.Chn: 4092 ->  3  n_pks: 197  Tot: 1800488.
284: SI2018/SI11418.Chn: 4108 ->  -13  n_pks: 175  Tot: 1451705.
285: SI2018/co2.chn: 4062 -

402: si2009/SI10909.Chn: 4111 ->  -16  n_pks: 243  Tot: 2389452.
403: si2009/filtro3.Chn: 3190 ->  905  n_pks: 34  Tot: 165679.
404: si2009/SI03509.Chn: 4102 ->  -7  n_pks: 190  Tot: 1641572.
405: si2009/SI12109.Chn: 4099 ->  -4  n_pks: 186  Tot: 1305017.
406: si2009/SI16909.Chn: 4106 ->  -11  n_pks: 159  Tot: 1395867.
407: si2009/SI07609.Chn: 4121 ->  -26  n_pks: 214  Tot: 1865139.
408: si2009/SI06409.Chn: 4113 ->  -18  n_pks: 187  Tot: 1551822.
409: si2009/SI09109.Chn: 4115 ->  -20  n_pks: 145  Tot: 1147636.
410: si2009/SI04609.Chn: 4088 ->  7  n_pks: 71  Tot: 387650.
411: si2009/SI14409.Chn: 4103 ->  -8  n_pks: 157  Tot: 1175087.
412: si2009/SI00809.Chn: 4115 ->  -20  n_pks: 182  Tot: 1617532.
413: si2009/si12609.Chn: 4041 ->  54  n_pks: 84  Tot: 1287836.
414: si2009/SI07209.Chn: 4119 ->  -24  n_pks: 202  Tot: 1785644.
415: si2009/si06309.Chn: 4102 ->  -7  n_pks: 195  Tot: 1769949.
416: si2009/SI14009.Chn: 4104 ->  -9  n_pks: 306  Tot: 3352030.
417: si2009/SI03909.Chn: 4109 ->  -14 

532: SI2022/SI04722.Chn: 4103 ->  -8  n_pks: 97  Tot: 570053.
533: SI2022/SI07722.Chn: 4111 ->  -16  n_pks: 96  Tot: 517142.
534: SI2022/SI06122.Chn: 4079 ->  16  n_pks: 95  Tot: 548003.
535: SI2022/SI05922.Chn: 4106 ->  -11  n_pks: 93  Tot: 544534.
536: SI2022/SI07922.Chn: 4111 ->  -16  n_pks: 108  Tot: 566099.
537: SI2022/SI09322.Chn: 4117 ->  -22  n_pks: 90  Tot: 556587.
538: SI2022/SI08722.Chn: 4111 ->  -16  n_pks: 101  Tot: 539942.
539: SI2022/SI00822.Chn: 4089 ->  6  n_pks: 51  Tot: 398225.
540: SI2022/SI01722.Chn: 4091 ->  4  n_pks: 80  Tot: 479312.
541: SI2022/SI03322.Chn: 4105 ->  -10  n_pks: 81  Tot: 496478.
542: SI2022/SI03122.Chn: 4094 ->  1  n_pks: 82  Tot: 492151.
543: SI2022/SI08222.Chn: 4110 ->  -15  n_pks: 99  Tot: 535351.
544: SI2022/SI02722.Chn: 4089 ->  6  n_pks: 10  Tot: 54990.
545: SI2022/SI02622.Chn: 4102 ->  -7  n_pks: 10  Tot: 49377.
546: SI2022/FRASCO ORIGINAL SI084_22.Chn: 4170 ->  -75  n_pks: 1  Tot: 2194.
547: SI2022/SI04022 (cont em 31-05-22).Chn: 4105 -> 

662: si2011/si11011.Chn: 4111 ->  -16  n_pks: 247  Tot: 2701598.
663: si2011/ERFGA.Chn: 4114 ->  -19  n_pks: 380  Tot: 38551654.
664: si2011/SI07611.Chn: 4106 ->  -11  n_pks: 167  Tot: 1373843.
665: si2011/SI00411.chn: 4091 ->  4  n_pks: 112  Tot: 816905.
666: si2011/SI12911.Chn: 4115 ->  -20  n_pks: 196  Tot: 2048920.
667: si2011/SI13711.Chn: 4115 ->  -20  n_pks: 179  Tot: 1744233.
668: si2011/SI10511.Chn: 4113 ->  -18  n_pks: 112  Tot: 779306.
669: si2011/SI15611.Chn: 4113 ->  -18  n_pks: 158  Tot: 1289901.
670: si2011/si04911.Chn: 4118 ->  -23  n_pks: 219  Tot: 2890836.
671: si2011/SI12311.Chn: 4104 ->  -9  n_pks: 119  Tot: 739287.
672: si2011/SI01211.Chn: 4113 ->  -18  n_pks: 138  Tot: 1146308.
673: si2011/SI05311.Chn: 4110 ->  -15  n_pks: 173  Tot: 1915442.
674: si2011/SI01711.Chn: 4097 ->  -2  n_pks: 101  Tot: 788580.
675: si2011/SI06111.Chn: 4104 ->  -9  n_pks: 250  Tot: 2644981.
676: si2011/SI08711.Chn: 4097 ->  -2  n_pks: 165  Tot: 1399530.
677: si2011/si07011.Chn: 4099 ->  -4

793: SI2004/SI18604.CHN: 4074 ->  21  n_pks: 173  Tot: 1236513.
794: SI2004/SI14004.CHN: 4075 ->  20  n_pks: 161  Tot: 1182815.
795: SI2004/si20404.CHN: 4094 ->  1  n_pks: 163  Tot: 1199142.
796: SI2004/si15204.CHN: 4116 ->  -21  n_pks: 272  Tot: 2574971.
797: si2013/sr90_CR.Chn: 4109 ->  -14  n_pks: 146  Tot: 1543925.
798: si2013/si05713.Chn: 4108 ->  -13  n_pks: 156  Tot: 1358116.
799: si2013/SI01913.Chn: 4101 ->  -6  n_pks: 154  Tot: 1480186.
800: si2013/si06413.Chn: 4113 ->  -18  n_pks: 192  Tot: 2251832.
801: si2013/SI03913.Chn: 4108 ->  -13  n_pks: 154  Tot: 1233031.
802: si2013/SI01113.Chn: 4108 ->  -13  n_pks: 191  Tot: 2204723.
803: si2013/si07913.Chn: 4108 ->  -13  n_pks: 187  Tot: 2156815.
804: si2013/si02713.Chn: 4116 ->  -21  n_pks: 147  Tot: 1227392.
805: si2013/SI06813.Chn: 4111 ->  -16  n_pks: 216  Tot: 2626817.
806: si2013/SI09013.Chn: 4132 ->  -37  n_pks: 177  Tot: 1702148.
807: si2013/si02813.Chn: 4115 ->  -20  n_pks: 158  Tot: 1376395.
808: si2013/SI02013.Chn: 4115 

926: Si2005/SI22205R.Chn: 4099 ->  -4  n_pks: 96  Tot: 667605.
927: Si2005/UNAT2.chn: 4083 ->  12  n_pks: 142  Tot: 964222.
928: Si2005/si06105.chn: 4083 ->  12  n_pks: 17  Tot: 77422.
929: Si2005/SI13105.CHN: 4084 ->  11  n_pks: 36  Tot: 181097.
930: Si2005/si18805.Chn: 4082 ->  13  n_pks: 25  Tot: 97842.
931: Si2005/si13005.CHN: 4084 ->  11  n_pks: 33  Tot: 189185.
932: Si2005/si14905.Chn: 4112 ->  -17  n_pks: 202  Tot: 1758298.
933: Si2005/SI04905.CHN: 4083 ->  12  n_pks: 20  Tot: 90857.
934: Si2005/si24105.Chn: 4113 ->  -18  n_pks: 189  Tot: 1464428.
935: Si2005/SI06805.CHN: 4109 ->  -14  n_pks: 153  Tot: 1076046.
936: Si2005/si19405b.Chn: 4111 ->  -16  n_pks: 647  Tot: 17409847.
937: Si2005/SI15405.Chn: 4039 ->  56  n_pks: 59  Tot: 1028391.
938: Si2005/si15505.chn: 4083 ->  12  n_pks: 51  Tot: 267823.
939: Si2005/Unat.CHN: 4083 ->  12  n_pks: 21  Tot: 82766.
940: Si2005/SI01705.CHN: 4071 ->  24  n_pks: 152  Tot: 1158002.
941: Si2005/si20305.Chn: 4085 ->  10  n_pks: 43  Tot: 221010

In [84]:
i_m

952

In [85]:
fig2 = go.Figure(data=go.Scattergl(
    x=x_nz,
    y=y_nz,
    mode='markers'
))


In [86]:
fig2.add_trace(
    go.Scattergl(x=x0_s,
                 y=ys_tot,
                 error_y=dict(
                     color='orange',width=3.0,
                     type='data', # value of error bar given in data coordinates
                     array=unc_y,
                     visible=True),
                 name="Counts & uncertaintes",
                 line=dict(color='orange',width=0.3)));

In [87]:
fig2.add_trace(
    go.Scattergl(x=pks,
                 y=y0_s[pks],
                 mode="markers",
                 name="peaks",
                 marker=dict(color='cyan',
                             symbol='circle',
                             size=15,
                             opacity=0.4,
                             line=dict(color='green', width=2.0)
                            )
                ));

In [88]:
# Set title and scale type    width=1800,

fig2.update_layout(
    title_text="Gamma-ray spectrum excerpt",
    height=900,
    margin=dict(l=10, r=10, t=30, b=10),
    paper_bgcolor="LightSteelBlue"
)
fig2.update_yaxes(type="log");
fig2.write_html('fig2.html', auto_open=True)


In [63]:
# Entendendo como funciona:
# a e b têm 21 canais cada
# O deslocamento é 4 (b vem 4 canais depois de a)
# Agora, usando y0_s e deslocado 479 canais à direita, vem:

In [64]:

a = y0_s
b = np.roll(y0_s, shift=479)

In [65]:
a_to_b = sci.signal.correlate(a,b)
b_to_a = sci.signal.correlate(b,a)
a_to_b_size = a_to_b.size
b_to_a_size = b_to_a.size
a_to_b_max = np.argmax(a_to_b)
b_to_a_max = np.argmax(b_to_a)

In [66]:
a_to_b

array([ 0,  0,  0, ..., 26, 36,  0])

In [67]:
b_to_a

array([ 0, 36, 26, ...,  0,  0,  0])

In [68]:
a_to_b_size

8191

In [69]:
b_to_a_size

8191

In [70]:
a_to_b_max

3616

In [71]:
b_to_a_max

4574

In [72]:
4574-3616

958

In [73]:
2*479

958

In [74]:
4095-3616

479

In [75]:
4095-4574

-479

In [76]:
# Die Ende